# File Operations: Basin, Met, Control, and Gage

This notebook demonstrates direct file operations for HMS component files:

| Class | File Type | Purpose |
|-------|-----------|--------|
| `HmsBasin` | `.basin` | Read/modify subbasins, reaches, junctions |
| `HmsMet` | `.met` | Read/modify gage assignments and precipitation methods |
| `HmsControl` | `.control` | Read/modify simulation time windows and intervals |
| `HmsGage` | `.gage` | Read gage definitions and DSS references |

These classes operate directly on HMS files without requiring project initialization, making them useful for batch processing and standalone file operations.

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

In [ ]:
from pathlib import Path
from hms_commander import (
    HmsExamples,
    init_hms_project,
    HmsBasin,
    HmsMet,
    HmsControl,
    HmsGage
)

print("hms-commander loaded")

## 1. Extract Example Project

We use the `castro` project which has multiple subbasins, reaches, and gages.

In [ ]:
# Extract the castro example project
project_path = HmsExamples.extract_project(
    "castro",
    output_path=Path.cwd() / 'hms_example_projects' / 'castro_file_ops'
)

print(f"Project extracted to: {project_path}")

# Initialize project for reference (optional - file ops work without this)
hms = init_hms_project(project_path)
print(f"Project: {hms.project_name}, Version: {hms.hms_version}")

## 2. Basin File Operations (HmsBasin)

`HmsBasin` provides methods to read and modify `.basin` files:
- `get_subbasins()` - List all subbasins with areas and methods
- `get_junctions()` - List all junctions
- `get_reaches()` - List all reaches with routing methods
- `get_loss_parameters()` / `set_loss_parameters()` - Loss method parameters
- `get_transform_parameters()` - Unit hydrograph parameters

In [ ]:
# Get the first basin file
basin_file = hms.basin_df['full_path'].iloc[0]
print(f"Basin file: {Path(basin_file).name}")
print("=" * 60)

### Read Subbasins

In [ ]:
# Get all subbasins from the basin file
subbasins = HmsBasin.get_subbasins(basin_file)

print(f"Found {len(subbasins)} subbasins:")
print("=" * 60)

# Display key columns
display_cols = ['name', 'area', 'downstream', 'loss_method', 'transform_method']
available_cols = [c for c in display_cols if c in subbasins.columns]
subbasins[available_cols]

### Read Junctions and Reaches

In [ ]:
# Get junctions
junctions = HmsBasin.get_junctions(basin_file)
print(f"Found {len(junctions)} junctions:")
print(junctions['name'].tolist())

# Get reaches
reaches = HmsBasin.get_reaches(basin_file)
print(f"\nFound {len(reaches)} reaches:")
display_cols = ['name', 'downstream', 'routing_method']
available_cols = [c for c in display_cols if c in reaches.columns]
display(reaches[available_cols])

### Read and Modify Loss Parameters

In [ ]:
# Get loss parameters for first subbasin
subbasin_name = subbasins['name'].iloc[0]
print(f"Loss parameters for '{subbasin_name}':")
print("=" * 60)

loss_params = HmsBasin.get_loss_parameters(basin_file, subbasin_name)
for key, value in loss_params.items():
    print(f"  {key}: {value}")

In [ ]:
# Modify loss parameters (example: update percent impervious)
# WARNING: This modifies the actual file! Changes persist to disk immediately.
# For QAQC workflows, consider using HmsBasin.clone_basin() first to preserve the original.
if 'percent_impervious' in loss_params:
    old_value = loss_params['percent_impervious']
    new_value = old_value + 5.0  # Increase by 5%

    HmsBasin.set_loss_parameters(
        basin_file,
        subbasin_name,
        percent_impervious=new_value
    )

    # Verify change
    updated_params = HmsBasin.get_loss_parameters(basin_file, subbasin_name)
    print(f"Updated percent_impervious: {old_value} -> {updated_params['percent_impervious']}")
    print("Note: This change is now persisted to the basin file.")
else:
    print("Subbasin does not use percent_impervious parameter")

### Read Transform Parameters

In [ ]:
# Get transform (unit hydrograph) parameters
print(f"Transform parameters for '{subbasin_name}':")
print("=" * 60)

transform_params = HmsBasin.get_transform_parameters(basin_file, subbasin_name)
for key, value in transform_params.items():
    print(f"  {key}: {value}")

## 3. Met File Operations (HmsMet)

`HmsMet` provides methods to read and modify `.met` files:
- `get_mets()` - List all met models in a file
- `get_precipitation_method()` - Get precip method type
- `get_gage_assignments()` - Get subbasin-to-gage mappings
- `set_gage_assignment()` - Update a subbasin's gage assignment
- `get_dss_references()` - Get DSS file references

In [ ]:
# Get the first met file
met_file = hms.met_df['full_path'].iloc[0]
print(f"Met file: {Path(met_file).name}")
print("=" * 60)

### Read Precipitation Method

In [ ]:
# Get precipitation method
met_name = hms.met_df['name'].iloc[0]
precip_method = HmsMet.get_precipitation_method(met_file, met_name)

print(f"Met model: {met_name}")
print(f"Precipitation method: {precip_method}")

### Read Gage Assignments

In [ ]:
# Get gage assignments (which gage is assigned to each subbasin)
# Returns: DataFrame with columns ['subbasin', 'precip_gage', 'weight']
assignments = HmsMet.get_gage_assignments(met_file, met_name)

print(f"Gage assignments for '{met_name}':")
print("=" * 60)
display(assignments)

### Read DSS References

In [ ]:
# Get DSS file references in the met model
dss_refs = HmsMet.get_dss_references(met_file, met_name)

print(f"DSS references in '{met_name}':")
print("=" * 60)
if dss_refs:
    for ref in dss_refs:
        print(f"  {ref}")
else:
    print("  No DSS references found (may use direct gage data)")

## 4. Control File Operations (HmsControl)

`HmsControl` provides methods to read and modify `.control` files:
- `get_controls()` - List all control specs in a file
- `get_time_window()` - Get start/end dates and times
- `set_time_window()` - Update simulation time window
- `get_time_interval()` - Get computation interval
- `set_time_interval()` - Update computation interval

In [ ]:
# Get the first control file
control_file = hms.control_df['full_path'].iloc[0]
print(f"Control file: {Path(control_file).name}")
print("=" * 60)

### Read Time Window

In [ ]:
# Get time window for the control spec
control_name = hms.control_df['name'].iloc[0]
time_window = HmsControl.get_time_window(control_file)

print(f"Time window for '{control_name}':")
print("=" * 60)
for key, value in time_window.items():
    print(f"  {key}: {value}")

### Read Time Interval

In [ ]:
# Get computation time interval
interval = HmsControl.get_time_interval(control_file)

print(f"Time interval for '{control_name}':")
print("=" * 60)
print(f"  Interval: {interval}")

### Get Full Control Info

In [ ]:
# Get complete control specification info
control_info = HmsControl.get_control_info(control_file)

print(f"Complete info for '{control_name}':")
print("=" * 60)
for key, value in control_info.items():
    print(f"  {key}: {value}")

## 5. Gage File Operations (HmsGage)

`HmsGage` provides methods to read `.gage` files:
- `get_gages()` - List all gages in a project
- `get_gage_info()` - Get detailed info for a specific gage
- `get_dss_pathname()` - Get DSS pathname for a gage
- `list_precip_gages()` / `list_discharge_gages()` - Filter by gage type

In [ ]:
# Get gage file path from project
gage_file = project_path / (project_path.name + ".gage")
print(f"Gage file: {gage_file.name}")
print(f"File exists: {gage_file.exists()}")
print("=" * 60)

### List All Gages

In [ ]:
# Get all gages from the gage file
# Note: If the gage file doesn't exist, we fall back to gage_df from project init.
# This silent fallback is intentional - HMS projects may store gage info in different files.
if gage_file.exists():
    gages = HmsGage.get_gages(str(gage_file))

    print(f"Found {len(gages)} gages:")
    print("=" * 60)

    display_cols = ['name', 'gage_type', 'dss_file']
    available_cols = [c for c in display_cols if c in gages.columns]
    display(gages[available_cols])
else:
    # Fallback: use project gage_df (parsed during project initialization)
    print(f"Gage file not found at: {gage_file}")
    print("Using project gage DataFrame (from hms.gage_df):")
    display(hms.gage_df[['name', 'gage_type', 'dss_file']])

### Get DSS Pathnames for Gages

In [ ]:
# Get DSS pathnames for all gages
print("DSS Pathnames:")
print("=" * 60)

for idx, row in hms.gage_df.iterrows():
    gage_name = row['name']
    gage_type = row['gage_type']
    
    # Try to get DSS pathname
    if 'dss_pathname' in row:
        pathname = row['dss_pathname']
    else:
        pathname = 'N/A'
    
    print(f"  {gage_name} ({gage_type}):")
    print(f"    {pathname}")

### Filter Gages by Type

In [ ]:
# Use project accessor methods to filter gages
precip_gages = hms.list_gage_names(gage_type='Precipitation')
flow_gages = hms.list_gage_names(gage_type='Flow')

print(f"Precipitation gages ({len(precip_gages)}): {precip_gages}")
print(f"Flow gages ({len(flow_gages)}): {flow_gages}")

## 6. Direct File Access (Without Project Initialization)

All file operations can be performed directly on files without initializing a project. This is useful for batch processing.

In [ ]:
# Example: Read basin elements directly from file path
# No init_hms_project() needed!

# Get basin filename dynamically from project DataFrame (recommended)
# Note: The column is 'file_name' (with underscore), not 'filename'
basin_filename = hms.basin_df['file_name'].iloc[0]
basin_path = project_path / basin_filename

if basin_path.exists():
    # Direct file read
    direct_subbasins = HmsBasin.get_subbasins(str(basin_path))
    direct_junctions = HmsBasin.get_junctions(str(basin_path))
    direct_reaches = HmsBasin.get_reaches(str(basin_path))

    print(f"Direct file access (no project init):")
    print(f"  Basin file: {basin_filename}")
    print(f"  Subbasins: {len(direct_subbasins)}")
    print(f"  Junctions: {len(direct_junctions)}")
    print(f"  Reaches: {len(direct_reaches)}")
else:
    print(f"Basin file not found at: {basin_path}")

## Summary

This notebook demonstrated file operations for the four main HMS file types:

| Class | Key Methods | Use Case |
|-------|-------------|----------|
| `HmsBasin` | `get_subbasins`, `get_loss_parameters`, `set_loss_parameters` | Modify basin parameters |
| `HmsMet` | `get_precipitation_method`, `get_gage_assignments` | Read met model config |
| `HmsControl` | `get_time_window`, `set_time_window` | Adjust simulation periods |
| `HmsGage` | `get_gages`, `get_dss_pathname` | Read gage definitions |

**Key Points**:
- All classes use **static methods** - no instantiation needed
- Operations work **directly on files** - project init is optional
- **Modifications persist** to disk immediately
- Useful for **batch processing** across multiple projects

## Next Steps

- **04_run_management.ipynb**: Configure simulation runs with validation
- **05_clone_workflow.ipynb**: Non-destructive cloning for QAQC
- **06_results_dss.ipynb**: Extract results from DSS files